In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("building_data/other.csv")
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
0,201901,153557,121.467843,25.107681,休閒公園,297183.166820,2.777786e+06
1,201901,153854,121.761409,25.031018,休閒公園,326838.776630,2.769429e+06
2,201901,153580,121.537519,24.971428,休閒公園,304270.221817,2.762720e+06
3,201901,433279,121.550270,24.974298,工業產業園區,305556.353766,2.763043e+06
4,201901,152189,121.523239,24.975946,休閒公園,302826.495936,2.763215e+06
...,...,...,...,...,...,...,...
4344,202401,189810,121.566753,25.031161,休閒公園,307194.211195,2.769349e+06
4345,202401,185484,121.581744,25.038115,休閒公園,308703.760315,2.770125e+06
4346,202401,191172,121.577324,25.031293,休閒公園,308260.950646,2.769368e+06
4347,202401,189990,121.559499,25.028643,休閒公園,306463.306745,2.769067e+06


In [10]:
print(df['CATEGORY'].unique())

['休閒公園' '工業產業園區' '殯葬設施' '垃圾場' '焚化廠']


In [11]:
park = df[df['CATEGORY'].isin(['休閒公園'])]
industry = df[df['CATEGORY'].isin(['工業產業園區'])]
bad = df[df['CATEGORY'].isin(['殯葬設施','垃圾場', '焚化廠'])]

In [12]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))
p_taipei

,x1,y1,x2,y2,cx,cy,index
0,296103,2777535,296203,2777635,296153,2777585,0
1,296103,2777635,296203,2777735,296153,2777685,1
2,296103,2777735,296203,2777835,296153,2777785,2
3,296103,2777835,296203,2777935,296153,2777885,3
4,296103,2777935,296203,2778035,296153,2777985,4
...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,317153,2768985,27656
27657,317103,2769035,317203,2769135,317153,2769085,27657
27658,317103,2769135,317203,2769235,317153,2769185,27658
27659,317103,2769235,317203,2769335,317153,2769285,27659


In [13]:
p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))
p_new_taipei

,x1,y1,x2,y2,cx,cy,index
0,278508,2777870,278608,2777970,278558,2777920,0
1,278508,2777970,278608,2778070,278558,2778020,1
2,278508,2778070,278608,2778170,278558,2778120,2
3,278508,2778170,278608,2778270,278558,2778220,3
4,278508,2778370,278608,2778470,278558,2778420,4
...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,351558,2767820,209743
209744,351608,2766970,351708,2767070,351658,2767020,209744
209745,351608,2767070,351708,2767170,351658,2767120,209745
209746,351608,2767270,351708,2767370,351658,2767320,209746


In [14]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [15]:
# 創建一個包含公園位置的GeoDataFrame
park_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(park['POI_X'], park['POI_Y'])])

park_100 = park_points1.buffer(100)
park_500 = park_points1.buffer(500)

park_100_gdf = gpd.GeoDataFrame(geometry=park_100, crs=park_points1.crs)
park_500_gdf = gpd.GeoDataFrame(geometry=park_500, crs=park_points1.crs)

In [16]:
# 創建一個包含工業園區位置的GeoDataFrame
industry_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(industry['POI_X'], industry['POI_Y'])])

industry_100 = industry_points.buffer(100)
industry_1000 = industry_points.buffer(1000)
industry_2000 = industry_points.buffer(2000)

industry_100_gdf = gpd.GeoDataFrame(geometry=industry_100, crs=industry_points.crs)
industry_1000_gdf = gpd.GeoDataFrame(geometry=industry_1000, crs=industry_points.crs)
industry_2000_gdf = gpd.GeoDataFrame(geometry=industry_2000, crs=industry_points.crs)

In [17]:
# 創建一個包含反向效果位置的GeoDataFrame
bad_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(bad['POI_X'], bad['POI_Y'])])

bad_100 = bad_points.buffer(100)
bad_500 = bad_points.buffer(500)
bad_1000 = bad_points.buffer(1000)
bad_2000 = bad_points.buffer(2000)

bad_100_gdf = gpd.GeoDataFrame(geometry=bad_100, crs=bad_points.crs)
bad_500_gdf = gpd.GeoDataFrame(geometry=bad_500, crs=bad_points.crs)
bad_1000_gdf = gpd.GeoDataFrame(geometry=bad_1000, crs=bad_points.crs)
bad_2000_gdf = gpd.GeoDataFrame(geometry=bad_2000, crs=bad_points.crs)

In [18]:
# 將百米方格加載為GeoDataFrame

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)
geometry_gdf_t


,geometry,index
0,"POLYGON ((296103.000 2777535.000, 296203.000 2...",0
1,"POLYGON ((296103.000 2777635.000, 296203.000 2...",1
2,"POLYGON ((296103.000 2777735.000, 296203.000 2...",2
3,"POLYGON ((296103.000 2777835.000, 296203.000 2...",3
4,"POLYGON ((296103.000 2777935.000, 296203.000 2...",4
...,...,...
27656,"POLYGON ((317103.000 2768935.000, 317203.000 2...",27656
27657,"POLYGON ((317103.000 2769035.000, 317203.000 2...",27657
27658,"POLYGON ((317103.000 2769135.000, 317203.000 2...",27658
27659,"POLYGON ((317103.000 2769235.000, 317203.000 2...",27659


In [19]:
def overlay_and_count(geometry_gdf, other_gdf, prefix):
    intersection = geometry_gdf.overlay(other_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_other_counts(geometry_gdf, park_100_gdf, park_500_gdf, industry_100_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf):
    park_100_cnt = overlay_and_count(geometry_gdf, park_100_gdf, 'park_100')
    park_500_cnt = overlay_and_count(geometry_gdf, park_500_gdf, 'park_500')
    industry_100_cnt = overlay_and_count(geometry_gdf, industry_100_gdf, 'industry_100')
    industry_1000_cnt = overlay_and_count(geometry_gdf, industry_1000_gdf, 'industry_1000')
    industry_2000_cnt = overlay_and_count(geometry_gdf, industry_2000_gdf, 'industry_2000')
    bad_100_cnt = overlay_and_count(geometry_gdf, bad_100_gdf, 'bad_100')
    bad_500_cnt = overlay_and_count(geometry_gdf, bad_500_gdf, 'bad_500')
    bad_1000_cnt = overlay_and_count(geometry_gdf, bad_1000_gdf, 'bad_1000')
    bad_2000_cnt = overlay_and_count(geometry_gdf, bad_2000_gdf, 'bad_2000')
    return park_100_cnt, park_500_cnt, industry_100_cnt, industry_1000_cnt, industry_2000_cnt, bad_100_cnt, bad_500_cnt, bad_1000_cnt, bad_2000_cnt

# 呼叫函式
park_100_cnt_t, park_500_cnt_t, industry_100_cnt_t, industry_1000_cnt_t, industry_2000_cnt_t, bad_100_cnt_t, bad_500_cnt_t, bad_1000_cnt_t, bad_2000_cnt_t = calculate_other_counts(geometry_gdf_t, park_100_gdf, park_500_gdf, industry_100_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf)
park_100_cnt_nt, park_500_cnt_nt, industry_100_cnt_nt, industry_1000_cnt_nt, industry_2000_cnt_nt, bad_100_cnt_nt, bad_500_cnt_nt, bad_1000_cnt_nt, bad_2000_cnt_nt = calculate_other_counts(geometry_gdf_nt, park_100_gdf, park_500_gdf, industry_100_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf)


In [20]:
def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['公園加權分數'] = (
        df['park_100_cnt'] * 1 + 
        df['park_500_cnt'] * 0.5
    )
    df['工業區加權分數'] = (
        df['industry_100_cnt'] * 1 + 
        df['industry_1000_cnt'] * 0.7 + 
        df['industry_2000_cnt'] * 0.3
    )
    df['反向效果加權分數'] = (
        df['bad_100_cnt'] * 1 + 
        df['bad_500_cnt'] * 0.8 + 
        df['bad_1000_cnt'] * 0.5 + 
        df['bad_2000_cnt'] * 0.3
    )
    
    return df

# 呼叫函式
result_t = calculate_weighted_score(p_taipei, park_100_cnt_t, park_500_cnt_t, industry_100_cnt_t, industry_1000_cnt_t, industry_2000_cnt_t, bad_100_cnt_t, bad_500_cnt_t, bad_1000_cnt_t, bad_2000_cnt_t)
result_nt = calculate_weighted_score(p_new_taipei, park_100_cnt_nt, park_500_cnt_nt, industry_100_cnt_nt, industry_1000_cnt_nt, industry_2000_cnt_nt, bad_100_cnt_nt, bad_500_cnt_nt, bad_1000_cnt_nt, bad_2000_cnt_nt)

In [22]:
result_t.sort_values('公園加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,park_100_cnt,park_500_cnt,industry_100_cnt,industry_1000_cnt,industry_2000_cnt,bad_100_cnt,bad_500_cnt,bad_1000_cnt,bad_2000_cnt,公園加權分數,工業區加權分數,反向效果加權分數
17144,307303,2765835,307403,2765935,307353,2765885,17144,7.0,26.0,0.0,0.0,0.0,0.0,3.0,7.0,33.0,20.0,0.0,15.8
2404,300103,2780835,300203,2780935,300153,2780885,2404,7.0,25.0,0.0,0.0,0.0,0.0,5.0,6.0,9.0,19.5,0.0,9.7
2463,300203,2769535,300303,2769635,300253,2769585,2463,7.0,25.0,0.0,0.0,0.0,0.0,2.0,10.0,38.0,19.5,0.0,18.0
2406,300103,2781035,300203,2781135,300153,2781085,2406,6.0,27.0,0.0,0.0,0.0,0.0,2.0,6.0,9.0,19.5,0.0,7.3
2405,300103,2780935,300203,2781035,300153,2780985,2405,7.0,25.0,0.0,0.0,0.0,0.0,2.0,6.0,9.0,19.5,0.0,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15485,306603,2786535,306703,2786635,306653,2786585,15485,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.6
15486,306603,2786635,306703,2786735,306653,2786685,15486,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.6
15487,306603,2786735,306703,2786835,306653,2786785,15487,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.6
15488,306603,2786835,306703,2786935,306653,2786885,15488,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.6


In [23]:
result_nt.sort_values('公園加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,park_100_cnt,park_500_cnt,industry_100_cnt,industry_1000_cnt,industry_2000_cnt,bad_100_cnt,bad_500_cnt,bad_1000_cnt,bad_2000_cnt,公園加權分數,工業區加權分數,反向效果加權分數
44422,295908,2772570,296008,2772670,295958,2772620,44422,6.0,21.0,0.0,2.0,6.0,0.0,0.0,4.0,19.0,16.5,3.2,7.7
65885,299508,2773970,299608,2774070,299558,2774020,65885,4.0,24.0,0.0,0.0,0.0,3.0,7.0,27.0,71.0,16.0,0.0,43.4
68128,299908,2773870,300008,2773970,299958,2773920,68128,7.0,17.0,0.0,0.0,0.0,1.0,10.0,31.0,70.0,15.5,0.0,45.5
43203,295708,2772470,295808,2772570,295758,2772520,43203,7.0,17.0,0.0,2.0,4.0,0.0,2.0,6.0,20.0,15.5,2.6,10.6
35224,294308,2784870,294408,2784970,294358,2784920,35224,8.0,15.0,0.0,0.0,2.0,3.0,4.0,12.0,19.0,15.5,0.6,17.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83810,302908,2796670,303008,2796770,302958,2796720,83810,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,7.0,0.0,0.0,3.9
83811,302908,2796770,303008,2796870,302958,2796820,83811,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,7.0,0.0,0.0,3.9
83812,302908,2796870,303008,2796970,302958,2796920,83812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,2.6
83820,303008,2730770,303108,2730870,303058,2730820,83820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 寫入csv
result_t.to_csv("taipei_other.csv", index=False)
result_nt.to_csv("new_taipei_other.csv", index=False)